In [1]:
from os import path
import sqlite3 as sql
import pandas as pd
import datetime as datetime

In [2]:
#Funcion para crear la Base de datos
def sql_connection(x,y):
    try:                                                 #intentara crear la base de datos
        con=sql.connect(x)                               #la creara la conexion
        cursor = con.cursor()
        cursor.execute("""CREATE TABLE IF NOT EXISTS "Transacciones" (
        "Fecha Transaccion" TEXT NOT NULL,
        "Bus" INTEGER NOT NULL,
        "Validador" INTEGER NOT NULL,
        "Tarjeta" INTEGER NOT NULL,
        "Tarifa-Monto" REAL NOT NULL,
        "C.Tipo Transaccion" INTEGER NOT NULL,
        "Fecha RegistroCle" TEXT NOT NULL,
        "Codigo Transaccion" INTEGER NOT NULL,
        "Fecha Contabilizacion" TEXT NOT NULL
        );""")
        con.commit()
        for i,row in y.iterrows():
            cursor.execute("""INSERT INTO Transacciones
            ("Fecha Transaccion",
            "Bus",
            "Validador",
            "Tarjeta",
            "Fecha RegistroCle",
            "Tarifa-Monto",
            "C.Tipo Transaccion",
            "Codigo Transaccion",
            "Fecha Contabilizacion") 
            VALUES( '"""+str(row['Fecha Transaccion'])+"',"
                       +str(row['Bus'])+","
                       +str(row['Validador'])+","
                       +str(row['Tarjeta'])+",'"
                       +str(row['Fecha RegistroCle'])+"',"
                       +str(row['Tarifa-Monto'])+",'"
                       +str(row['C.Tipo Transaccion'])+"',"
                       +str(row['Codigo Transaccion'])+",'"
                       +str(row['Fecha Contabilizacion'])+"');")
            con.commit()
            
        Rutas.to_sql('Rutas',con,index=False,if_exists = 'replace')#Transforma el archivo csv en Base de datos alternativa no valida
        print("Base de Datos Creada")             #Imprime si esta lista
    except Exception as e:                                        #de existir marcara error
        print("La Base de Datos no fue creada debido a: "+ e)
    finally:                                             #simplemente cerrara la conexion una vez salga
        con.close()


In [3]:
#Funcion para crear la Base de datos
def sql_connection2(x,y):
    try:                                                 #intentara crear la base de datos
        con=sql.connect(x)                               #la creara la conexion
        cursor = con.cursor()
        y.to_sql('Transacciones',con,index=False,if_exists = 'replace')#Transforma el archivo csv en Base de datos alternativa no valida
        print("Base de Datos Creada")             #Imprime si esta lista
    except Exception as e:                                        #de existir marcara error
        print("La Base de Datos no fue creada debido a: "+ e)
    finally:                                             #simplemente cerrara la conexion una vez salga
        con.close()


In [4]:
#Vista 1.	¿Cuántas transacciones se realizan por cada hora? (mostrar las 24 horas)
def Crearlasvista(x):
    con=sql.connect(x)                               #la creara la conexion
    cursor = con.cursor()
    
    #1.	¿Cuántas transacciones se realizan por cada hora? (mostrar las 24 horas)
    cursor.execute("""CREATE VIEW IF NOT EXISTS Vista0 as
                      SELECT
                      strftime('%H',"Fecha Transaccion") as Hora,
                      COUNT(strftime('%H',"Fecha Transaccion")) as "Cantidad de Transacciones"
                      FROM  Transacciones 
                      where strftime('%H',"Fecha Transaccion") 
                      group by strftime('%H',"Fecha Transaccion");""")  
    con.commit()
    #Vista 2.	¿Cuántos pasajeros se movilizaron por cada hora?
    cursor.execute("""CREATE VIEW IF NOT EXISTS Vista1 as
                      SELECT                strftime('%H',"Fecha Transaccion") as Hora,
                      COUNT("Codigo Transaccion") as "Pasajeros Entrantes"
                      FROM  Transacciones 
                      where strftime('%H',"Fecha Transaccion") AND "Codigo Transaccion"= 19
                      group by strftime('%H',"Fecha Transaccion");""")  
    con.commit()
    
    #Vista 3.	¿Cuántos buses operaron por cada hora?
    cursor.execute("""CREATE VIEW IF NOT EXISTS Vista2 as
                      SELECT strftime('%H',"Fecha Transaccion") as Hora,
                      COUNT(Bus) as "Autobuses por Hora" 
                      FROM  Transacciones 
                      where strftime('%H',"Fecha Transaccion") 
                      group by strftime('%H',"Fecha Transaccion");""")  
    con.commit()
    
    #Vista 4.	¿Cuáles son los 30 buses que más pasajeros transportaron?
    cursor.execute("""CREATE VIEW IF NOT EXISTS Vista3 as
                      SELECT                Bus as "Hora",
                      COUNT("Codigo Transaccion") as "Pasajeros Entrantes"
                      FROM  Transacciones 
                      group by Bus 
                      ORDER BY COUNT("Codigo Transaccion") DESC 
                      LIMIT 30;
                      """)  
    con.commit()
    
    #Vista 5.	¿Cuál fue el promedio de ingresos por hora?
    cursor.execute("""CREATE VIEW IF NOT EXISTS Vista4 as
                      SELECT
                      strftime('%H',"Fecha Transaccion") as Hora,
                      COUNT(strftime('%H',"Fecha Transaccion")) as "Cantidad de Transacciones"
                      FROM  Transacciones 
                      where strftime('%H',"Fecha Transaccion") 
                      group by strftime('%H',"Fecha Transaccion");""")  
    con.commit()
    
    #Vista 6.	¿Qué buses fueron los que más ingresos generaron?
    cursor.execute("""CREATE VIEW IF NOT EXISTS Vista5 as
                      select Bus, COUNT("Tarifa-Monto") as "Venta Total Por Buses" FROM Transacciones
                      group by Bus
                      order by sum("Tarifa-Monto")desc; """)  
    con.commit()
    
    #Vista 7.	¿Cuáles fueron los ingresos durante el día en intervalos de quince minutos?
    cursor.execute("""CREATE VIEW IF NOT EXISTS Vista6 as
                      SELECT DISTINCT strftime('%H:%M',"Fecha Transaccion") as Intervalos,
	   count("Codigo Transaccion") as "Personas ingresando"
       FROM  Transacciones
	   group by strftime('%H:%M',"Fecha Transaccion")
	   order by strftime('%H:%M',"Fecha Transaccion") DESC""")  
    con.commit()
    

In [5]:
#Declaraciones
pathcsv="./Data/transacciones.csv"                             #filepath para la Conexion con el archivo Crudo
pathdb="./DBSQL/transacciones.db"                               #Ruta de la base de datos 
pathlimpio="./Problema_2/transacciones.csv"
path=["./Problema_2/Transxhora.csv","./Problema_2/Pasajeroxhora.csv","./Problema_2/Busesxhora.csv","./Problema_2/TOP30Buses.csv","./Problema_2/Promediodeingresos.csv","./Problema_2/BusesIngresos.csv","./Problema_2/Intervalos15Min.csv"]
Nombredetable="Transacciones"

In [6]:
#Limpieza de Datos
Dataframe = pd.read_csv(pathcsv,encoding = "ISO-8859-1")
Dataframe.dropna()
#Corrige Fechas
Dataframe["Fecha Transaccion"]= pd.to_datetime(Dataframe["Fecha Transaccion"])
Dataframe["Fecha RegistroCle"]= pd.to_datetime(Dataframe["Fecha RegistroCle"])
Dataframe["Fecha Contabilizacion"]= pd.to_datetime(Dataframe["Fecha Contabilizacion"])

In [7]:
#Creacion de Base de Datos y Tablas
sql_connection2(pathdb,Dataframe)

C:\Users\PJ Roberto\anaconda3\lib\site-packages\pandas\core\generic.py:2653: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


Base de Datos Creada


In [8]:
#Creacion de Vista Enunciado 1
Crearlasvista(pathdb)

In [9]:
#Creacion de CSV
con=sql.connect(pathdb)  
#Exportacion de Datos en Limpio
Dataframe = pd.read_sql_query("SELECT * FROM "+Nombredetable+";",con)  #Transforma La vista en un Dataframe
Dataframe.to_csv(pathlimpio,index=False)    #Coloca el Archivo CSV en la ruta selecionada sin la indexacion

for index, path in enumerate(path):
    df = pd.read_sql_query("SELECT * FROM Vista"+str(index)+";",con) 
    df.to_csv(path,index=False)
